In [1]:
import Prelude ()
import qualified Prelude
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold.Types
import Data.LinearMap.HerMetric
import Data.VectorSpace
import Data.Semigroup
import Data.Foldable (toList)
import Control.Monad (replicateM)

import Control.Category.Constrained.Prelude
:opt no-lint  -- lint gives bogus warnings with constrained-categories

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2

In [3]:
convolveShade' (0|±|[1]) (0|±|[1]) :: Shade' ℝ

Shade' {_shade'Ctr = 0.0, _shade'Narrowness = projector 0.5}

In [4]:
convolveShade' ((0,0)|±|[(1,0),(0,1)]) ((0,0)|±|[(99,0),(0,99)]) :: Shade' ℝ²

Shade' {_shade'Ctr = (0.0,0.0), _shade'Narrowness = projector (0.0,9.999999999999998e-3) ^+^ projector (9.999999999999998e-3,0.0)}

In [5]:
convoTests :: [[Shade' ℝ²]]
convoTests = [ [ (0,3)|±|[(2,0),(0,1)]
               , (5,2)|±|[(2,0),(0,1)] ]
             , [ (8,0)|±|[(4,0),(0,1)]
               , (0,8)|±|[(1,0),(0,4)] ]
             , [ (8,0)|±|[(3,4),(-0.3,1)]
               , (0,8)|±|[(1,-0.3),(4,3)] ] ]

In [25]:
forM_ convoTests $ \wings -> do
  let convo = convolveShade' (wings!!0) (wings!!1)
  
  testPts <- runRVar (forM wings $ \sh@(Shade' cs es) ->
    fmap (filter $ \p -> occlusion sh p > exp(-1))
     . replicateM 200 $ do
        φ <- uniform 0 (2*pi)
        r <- normal 1 0.1
        let [v₁,v₂] = eigenCoSpan' es
        return $ cs ^+^ (r*cos φ)*^v₁ ^+^ (r*sin φ)*^v₂
   ) StdRandom  :: IO [[ℝ²]]

  plotWindow $ [ plot $ plot sh : [plot $ (p::ℝ²)|±|[(0.01,0),(0,0.01)] | p<-tps::[ℝ²]]
               | (sh,tps) <- zip (convo:wings) ([a^+^b | a<-testPts!!0, b<-testPts!!1] : testPts)]
            ++ (plot<$>[xInterval (-2,8), yInterval (-1,7)])

![Convolution of coaligned shades](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ShadeCombinations/2Dconvolution-coaligned.png)
![Convolution of "orthogonal" shades](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ShadeCombinations/2Dconvolution-orthogonal.png)
![Convolution of skewed shades](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ShadeCombinations/2Dconvolution-skewed.png)